In [18]:


#Import necessary packages and modules 

import yfinance as yf
import feedparser
import pandas as pd
import pandas_datareader.data as web

import datetime
import numpy as np


import modal
import csv
import os

[nltk_data] Downloading package vader_lexicon to C:\Users\Marlon
[nltk_data]     Dominguez\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
#First retrieve compnies in S&P500 

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

#The first table has the company abbreviations
sp500_tickers = tables[0]['Symbol'].tolist()

#Show the companies 
for ticker in sp500_tickers:
    print(ticker)



MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
APO
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BAX
BDX
BRK.B
BBY
TECH
BIIB
BLK
BX
BK
BA
BKNG
BSX
BMY
AVGO
BR
BRO
BF.B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CAT
CBOE
CBRE
CDW
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DELL
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DASH
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ERIE
ESS
EL
EG
EVRG
ES
EXC
EXE
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM
H

In [20]:
#CUse Modal app cloud computing services 
app = modal.App("Retrieving and Storing Stock Data")

#Create Modal volume called my-volume
volume = modal.Volume.from_name("my-volume", create_if_missing=True)

image = modal.Image.debian_slim().pip_install(
    "yfinance", "feedparser", "nltk", "pandas", "numpy", "pandas_datareader"
)

#generate CSV files
@app.function(
    image=image,
    cpu=2,
    memory=4096,
    timeout=5000,
    volumes={"/my_vol": volume}  # Mount the volume to /my_vol
)
async def generate_csv_files(sp500_tickers):
    import pandas as pd
    import pandas_datareader.data as web
    import yfinance as yf
    import numpy as np
    
    nltk.download('vader_lexicon') 
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    csv_files = []

    for ticker in sp500_tickers:
        try:
            # Download stock data for the company
            filename = f"/my_vol/{ticker}_30_years.csv"  # Save inside mounted volume
            
            data = yf.download(ticker, start="2015-03-28", end="2024-03-28")

            #Alot of preprocessing before feeding data into model 
            # Add additional features to be used
            data['News_Sentiment'] = None 
            data['Economic_Health'] = None

            
            data.to_csv(filename)
            # Re-read the CSV file to adjust the column names if needed
            df = pd.read_csv(filename, skiprows=2)
            df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'News_Sentiment', 'Economic Health']
            
            # Save the modified data back to CSV in the volume
            df.to_csv(filename, index=False)

            # Append the cleaned filename to the list
            csv_files.append(filename)
        
            print(f"Data for {ticker} saved as {filename}")

        except Exception as e:
            print(f"Error downloading data for {ticker}: {e}")

    # Commit changes to the volume to ensure the files are persisted
    volume.commit()

    return csv_files

#Run the app to generate files
with modal.enable_output():
    with app.run():
        print("Sentiment Analysis is running...")  # Keeps the app active
        
        try:
            
            # Generate the CSV files by calling the remote function
            csv_files = generate_csv_files.remote(sp500_tickers)
            print(f"Generated files: {csv_files}")  # List the generated files inside the volume

        except Exception as e:
            print(f"Error while fetching result: {e}")

Output()

✓ Initialized. View run at https://modal.com/apps/marlon-dominguez307/main/ap-Q6BezDxsNvLEiNw4DFFgMT

Output()

✓ Created objects.
└── 🔨 Created function generate_csv_files.

Output()

...

[*********************100%***********************]  1 of 1 completed


Data for ZTS saved as /my_vol/ZTS_30_years.csv


Generated files: ['/my_vol/MMM_30_years.csv', '/my_vol/AOS_30_years.csv', '/my_vol/ABT_30_years.csv', 
'/my_vol/ABBV_30_years.csv', '/my_vol/ACN_30_years.csv', '/my_vol/ADBE_30_years.csv', '/my_vol/AMD_30_years.csv', 
'/my_vol/AES_30_years.csv', '/my_vol/AFL_30_years.csv', '/my_vol/A_30_years.csv', '/my_vol/APD_30_years.csv', 
'/my_vol/ABNB_30_years.csv', '/my_vol/AKAM_30_years.csv', '/my_vol/ALB_30_years.csv', '/my_vol/ARE_30_years.csv', 
'/my_vol/ALGN_30_years.csv', '/my_vol/ALLE_30_years.csv', '/my_vol/LNT_30_years.csv', '/my_vol/ALL_30_years.csv', 
'/my_vol/GOOGL_30_years.csv', '/my_vol/GOOG_30_years.csv', '/my_vol/MO_30_years.csv', '/my_vol/AMZN_30_years.csv', 
'/my_vol/AMCR_30_years.csv', '/my_vol/AEE_30_years.csv', '/my_vol/AEP_30_years.csv', '/my_vol/AXP_30_years.csv', 
'/my_vol/AIG_30_years.csv', '/my_vol/AMT_30_years.csv', '/my_vol/AWK_30_years.csv', '/my_vol/AMP_30_years.csv', 
'/my_vol/AME_30_years.csv', '/my_vol/AMGN_30_years.csv', '/my_vol/APH_30_years.csv', '/my_vol/ADI_30_years.csv', 
'/my_vol/ANSS_30_years.csv', '/my_vol/AON_30_years.csv', '/my_vol/APA_30_years.csv', '/my_vol/APO_30_years.csv', 
'/my_vol/AAPL_30_years.csv', '/my_vol/AMAT_30_years.csv', '/my_vol/APTV_30_years.csv', '/my_vol/ACGL_30_years.csv',
'/my_vol/ADM_30_years.csv', '/my_vol/ANET_30_years.csv', '/my_vol/AJG_30_years.csv', '/my_vol/AIZ_30_years.csv', 
'/my_vol/T_30_years.csv', '/my_vol/ATO_30_years.csv', '/my_vol/ADSK_30_years.csv', '/my_vol/ADP_30_years.csv', 
'/my_vol/AZO_30_years.csv', '/my_vol/AVB_30_years.csv', '/my_vol/AVY_30_years.csv', '/my_vol/AXON_30_years.csv', 
'/my_vol/BKR_30_years.csv', '/my_vol/BALL_30_years.csv', '/my_vol/BAC_30_years.csv', '/my_vol/BAX_30_years.csv', 
'/my_vol/BDX_30_years.csv', '/my_vol/BBY_30_years.csv', '/my_vol/TECH_30_years.csv', '/my_vol/BIIB_30_years.csv', 
'/my_vol/BLK_30_years.csv', '/my_vol/BX_30_years.csv', '/my_vol/BK_30_years.csv', '/my_vol/BA_30_years.csv', 
'/my_vol/BKNG_30_years.csv', '/my_vol/BSX_30_years.csv', '/my_vol/BMY_30_years.csv', '/my_vol/AVGO_30_years.csv', 
'/my_vol/BR_30_years.csv', '/my_vol/BRO_30_years.csv', '/my_vol/BLDR_30_years.csv', '/my_vol/BG_30_years.csv', 
'/my_vol/BXP_30_years.csv', '/my_vol/CHRW_30_years.csv', '/my_vol/CDNS_30_years.csv', '/my_vol/CZR_30_years.csv', 
'/my_vol/CPT_30_years.csv', '/my_vol/CPB_30_years.csv', '/my_vol/COF_30_years.csv', '/my_vol/CAH_30_years.csv', 
'/my_vol/KMX_30_years.csv', '/my_vol/CCL_30_years.csv', '/my_vol/CARR_30_years.csv', '/my_vol/CAT_30_years.csv', 
'/my_vol/CBOE_30_years.csv', '/my_vol/CBRE_30_years.csv', '/my_vol/CDW_30_years.csv', '/my_vol/COR_30_years.csv', 
'/my_vol/CNC_30_years.csv', '/my_vol/CNP_30_years.csv', '/my_vol/CF_30_years.csv', '/my_vol/CRL_30_years.csv', 
'/my_vol/SCHW_30_years.csv', '/my_vol/CHTR_30_years.csv', '/my_vol/CVX_30_years.csv', '/my_vol/CMG_30_years.csv', 
'/my_vol/CB_30_years.csv', '/my_vol/CHD_30_years.csv', '/my_vol/CI_30_years.csv', '/my_vol/CINF_30_years.csv', 
'/my_vol/CTAS_30_years.csv', '/my_vol/CSCO_30_years.csv', '/my_vol/C_30_years.csv', '/my_vol/CFG_30_years.csv', 
'/my_vol/CLX_30_years.csv', '/my_vol/CME_30_years.csv', '/my_vol/CMS_30_years.csv', '/my_vol/KO_30_years.csv', 
'/my_vol/CTSH_30_years.csv', '/my_vol/CL_30_years.csv', '/my_vol/CMCSA_30_years.csv', '/my_vol/CAG_30_years.csv', 
'/my_vol/COP_30_years.csv', '/my_vol/ED_30_years.csv', '/my_vol/STZ_30_years.csv', '/my_vol/CEG_30_years.csv', 
'/my_vol/COO_30_years.csv', '/my_vol/CPRT_30_years.csv', '/my_vol/GLW_30_years.csv', '/my_vol/CPAY_30_years.csv', 
'/my_vol/CTVA_30_years.csv', '/my_vol/CSGP_30_years.csv', '/my_vol/COST_30_years.csv', '/my_vol/CTRA_30_years.csv',
'/my_vol/CRWD_30_years.csv', '/my_vol/CCI_30_years.csv', '/my_vol/CSX_30_years.csv', '/my_vol/CMI_30_years.csv', 
'/my_vol/CVS_30_years.csv', '/my_vol/DHR_30_years.csv', '/my_vol/DRI_30_years.csv', '/my_vol/DVA_30_years.csv', 
'/my_vol/DAY_30_years.csv', '/my_vol/DECK_30_years.csv', '/my_vol/DE_30_years.csv', '/my_vol/DELL_30_years.csv', 
'/my_vol/DAL_30_years.csv', 

Stopping app - local entrypoint completed.


✓ App completed. View run at https://modal.com/apps/marlon-dominguez307/main/ap-Q6BezDxsNvLEiNw4DFFgMT

In [50]:
#Test example of assigning sentiment values to articles

import nltk
nltk.download('vader_lexicon') 
from nltk.sentiment.vader import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()
news_headline = "Stock market surges as Fed signals lower interest rates."

for entry in feed.entries[:5]:  # Show first 5 news articles as example
    
    print(entry.title)
    
    sentiment = analyzer.polarity_scores(entry.title)
    print(sentiment)  # {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.658}
    print()

[nltk_data] Downloading package vader_lexicon to C:\Users\Marlon
[nltk_data]     Dominguez\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


NameError: name 'feed' is not defined

In [50]:
#Using sentiment analysis for news article

import asyncio

# Initialize Modal App
app = modal.App("Stock Data with Sentiment Analysis - Persistent Storage")

#Use Docker Image with dependencies
image = modal.Image.debian_slim().pip_install(
    "yfinance", "feedparser", "nltk", "pandas", "numpy"
)

#Correct way to initialize a persisted volume
volume = modal.Volume.from_name("my-volume")

#Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

@app.function(
    image=image,
    volumes={"/my_vol": volume},
    cpu=10,  # More CPU for sentiment analysis
    memory=4000,
    timeout=20000
)

async def fetch_sentiment(ticker, date):
    """
    Fetches news articles related to the ticker on a specific date,
    performs sentiment analysis, and returns an average sentiment score.
    """
    try:
        url = f"https://news.google.com/rss/search?q={ticker}+stock+market+news+{date}"
        feed = feedparser.parse(url)

        news_sentiments = []
        
        if feed.entries:
            for entry in feed.entries[:5]:  # Limit to 5 articles
                sentiment_score = analyzer.polarity_scores(entry.title)["compound"]
                
                # Categorize sentiment
                sentiment = (
                    -1 if sentiment_score <= -0.05 else
                    1 if sentiment_score >= 0.05 else
                    0
                )

                news_sentiments.append(sentiment)
        else: news_sentiments.append(0)
        # Compute average sentiment score if valid sentiments exist
        return np.mean(news_sentiments) if news_sentiments else None

    except Exception as e:
        import traceback
        print(f"Error processing {ticker} on {date}: {e}")
        traceback.print_exc()
        return None

@app.function(
    image=image,
    volumes={"/my_vol": volume},
    cpu=15,
    memory=10000,
    timeout=20000
)
async def process_ticker(ticker):
    """
    Downloads stock data for a given ticker, updates it with sentiment scores,
    and saves it to the Modal volume.
    """
    try:
        filename = f"/my_vol/{ticker}_30_years.csv"  # Path to the volume
        print(f"Processing ticker {ticker}")

        # List files in the mounted volume to check if the file exists
        existing_files = os.listdir("/my_vol")  # Get the list of files in the volume
        if f"{ticker}_30_years.csv" in existing_files:
            df = pd.read_csv(filename, index_col="Date", parse_dates=True)
            print(f"File {filename} found. Loading data...")
        else:
            # If the file doesn't exist, download the stock data and save it to the volume
            df = yf.download(ticker, start="2015-03-25", end="2024-03-28")
            df["News_Sentiment"] = None  # Initialize empty column for sentiment
            print(f"File {filename} not found. Downloading stock data...")

        # Process missing sentiment values concurrently
        tasks = []
        for idx, row in df.iterrows():
            date = idx.date()  # Correctly extract the date from index

            # Fetch sentiment if not already present
            if pd.isna(row["News_Sentiment"]):
                print(f"Fetching sentiment for {ticker} on {date}")
                task = fetch_sentiment.remote(ticker, date)  # Fetch sentiment concurrently
                tasks.append(task)
        
        # Wait for all tasks to finish and update DataFrame with the sentiment results
        results = await modal.TaskGroup(tasks).results()

        # Update DataFrame with sentiment scores
        for idx, sentiment_score in zip(df.index, results):
            df.at[idx, "News_Sentiment"] = sentiment_score

        # Save updated stock data with sentiment analysis to the volume
        df.to_csv(filename)
        print(f"Updated data for {ticker} saved in {filename}")

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        import traceback
        traceback.print_exc()

@app.function(
    image=image,
    volumes={"/my_vol": volume}
)
def list_csv_files():
    """ Lists all CSV files stored in the volume. """
    try:
        return os.listdir("/my_vol/")
    except Exception as e:
        print(f"Error listing files in volume: {e}")
        return []

#Run the Modal App
with modal.enable_output():
    with app.run():
        print("Sentiment Analysis is running...")

        # List of tickers to process concurrently
         # Example tickers
        
        # Create a TaskGroup to process all tickers concurrently
        tasks = [process_ticker.remote(ticker) for ticker in sp500_tickers]
        
        # Wait for all tickers to be processed
        await modal.TaskGroup(tasks).results()

        # List stored files after processing
        stored_files = list_csv_files.remote()
        print("Files stored in volume:", stored_files)

Output()

✓ Initialized. View run at https://modal.com/apps/marlon-dominguez307/main/ap-V5gfWXvp7cYeoUM9zoh3D1

Output()

C:\Users\Marlon Dominguez\AppData\Roaming\Python\Python310\site-packages\modal\_functions.py:731: UserWarning: 
Function <function fetch_sentiment at 0x00000259E33860E0> has size 588891 bytes when packaged. This is larger than 
the recommended limit of 256 KiB. Try reducing the size of the closure by using parameters or mounts, not large 
global variables.
  warnings.warn(

✓ Created objects.
├── 🔨 Created function fetch_sentiment.
├── 🔨 Created function process_ticker.
└── 🔨 Created function list_csv_files.

...

Stopping app - user stopped from dashboard.


Traceback (most recent call last):


Error processing GOOG: 


  File "C:\Users\Marlon Dominguez\AppData\Local\Temp\ipykernel_20920\4072951726.py", line 97, in process_ticker
  File "/pkg/synchronicity/synchronizer.py", line 592, in proxy_method
    return wrapped_method(instance, *args, **kwargs)
  File "/pkg/synchronicity/combined_types.py", line 29, in __call__
    raise uc_exc.exc
  File "/pkg/modal/_object.py", line 286, in wrapped
    return await method(self, *args, **kwargs)
  File "/pkg/modal/_functions.py", line 1387, in remote
    return await self._call_function(args, kwargs)
  File "/pkg/modal/_functions.py", line 1337, in _call_function
    return await invocation.run_function()
  File "/pkg/modal/_functions.py", line 256, in run_function
    return await self._get_single_output()
  File "/pkg/modal/_functions.py", line 244, in _get_single_output
    return await _process_result(item.result, item.data_format, self.stub, self.client)
  File "/pkg/modal/_utils/function_utils.py", line 497, in _process_result
    raise RemoteError(result.exception)
modal.exception.RemoteError


RemoteError: 

In [40]:
#Generate economic health coefficient 

#Load File
gdp_df = pd.read_csv('Economy_Info/GDP.csv', skiprows=2)
unemployment_df = pd.read_csv('Economy_Info/Unemployment.csv', skiprows=2)
inflation_df = pd.read_csv('Economy_Info/Inflation.csv', skiprows=2)
poverty_df = pd.read_csv('Economy_Info/Poverty.csv', skiprows=2)

#Function to extract data for the United States
def get_us_data(df):
    # Filter the data for the United States
    us_data = df[df['Country Name'] == 'United States'].iloc[:, 4:].values.flatten()
    return us_data

#Get US data
gdp_us = get_us_data(gdp_df)
unemployment_us = get_us_data(unemployment_df)
inflation_us = get_us_data(inflation_df)
poverty_us = get_us_data(poverty_df)

# Make NULL values with a default value (I prefer the median!)
#This is for cleaning the data
#Remember, if we can keep the data clean at the root, it will grow to be clean aswell
def fill_missing_data(data, method='mean'):
    if method == 'mean':
        return np.nan_to_num(data, nan=np.nanmean(data))
    elif method == 'median':
        return np.nan_to_num(data, nan=np.nanmedian(data))
    else:
        return np.nan_to_num(data, nan=0)

# Fill missing data in all arrays
gdp_us = fill_missing_data(gdp_us)
unemployment_us = fill_missing_data(unemployment_us)
inflation_us = fill_missing_data(inflation_us)
poverty_us = fill_missing_data(poverty_us)

# Store the data in arrays
data_arrays = {
    'GDP': gdp_us,
    'Unemployment': unemployment_us,
    'Inflation': inflation_us,
    'Poverty': poverty_us
}

#Normalize the data (USING MEDIAN)
def normalize(data):
    # Normalize the data (handling NaN values)
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val)

#Normalize all the data
normalized_gdp = normalize(gdp_us)
normalized_unemployment = normalize(unemployment_us)
normalized_inflation = normalize(inflation_us)
normalized_poverty = normalize(poverty_us)
years = gdp_df.columns[4:].values  # Extract years from the column names of the DataFrame
years_filtered = [year for year in years if int(year) >= 2015]

# Get indices of the filtered years
filtered_indices = [i for i, year in enumerate(years) if int(year) >= 2015]

# Filter the data to include only the years 2020 and beyond
filtered_gdp = normalized_gdp[filtered_indices]
filtered_unemployment = normalized_unemployment[filtered_indices]
filtered_inflation = normalized_inflation[filtered_indices]
filtered_poverty = normalized_poverty[filtered_indices]

# Calculate the Economic Health Score for each year
# Economic Health = 0.25 * GDP + 0.25 * (1 - Unemployment) + 0.25 * (1 - Inflation) + 0.25 * (1 - Poverty)

economic_health_yearly = 0.25 * filtered_gdp + 0.25 * (1 - filtered_unemployment) + 0.25 * (1 - filtered_inflation) + 0.25 * (1 - filtered_poverty)

# Store the economic health values in a DataFrame for better visualization
economic_health_df = pd.DataFrame({
    'Year': years_filtered,
    'Economic Health': economic_health_yearly
})

#Print the resulting DataFrame with the Economic Health Values for each year
print("Economic Health Values for each year (United States) from 2020 onward:")
print(economic_health_df)

Economic Health Values for each year (United States) from 2020 onward:
   Year  Economic Health
0  2015         0.586300
1  2016         0.637580
2  2017         0.600823
3  2018         0.673953
4  2019         0.702880
5  2020         0.728671
6  2021         0.800634
7  2022         0.583457
8  2023         0.781265


In [22]:
#Append the health coefficient to dataset 
# Initialize Modal App
# Economic health values (as a list)
economic_health_values = [
    0.586300, 0.637580, 0.600823, 0.673953, 0.702880, 
    0.728671, 0.800634, 0.583457, 0.781265
]

app = modal.App("Stock Data with Random Sentiment Update")

# Mount the existing volume


# Define the function to update sentiment
@app.function(
    image=modal.Image.debian_slim().pip_install("pandas", "numpy"),
    volumes={"/my_vol": volume}
)
def update_random_economic_health_in_volume(ticker):
    try:
        filename = f"/my_vol/{ticker}_30_years.csv"

        # Load the file from Modal volume
        if f"{ticker}_30_years.csv" in os.listdir("/my_vol"):
            # Read the CSV file into a DataFrame
        
            print(f"Loaded {ticker} data successfully.")

            df = pd.read_csv(filename, index_col="Date", parse_dates=True)
            print(f"Loaded {ticker} data successfully.")
            
            # Assign a random economic health value to the 'Economic Health' column
            df['Economic Health'] = np.random.choice(economic_health_values, size=len(df))
            
            # Save the updated DataFrame back to the volume
            df.to_csv(filename)
            print(f"Updated {ticker} with random economic health values and saved to volume.")

        else:
            print(f"File for {ticker} not found in volume.")

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Run the Modal App
with app.run():
    tickers = sp500_tickers # Example tickers, replace with your actual tickers
    for ticker in tickers:
        update_random_economic_health_in_volume.remote(ticker)

In [29]:
#all data are in and we are ready to train model. 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib  # For saving and loading the scaler
import modal


image = modal.Image.debian_slim().pip_install("torch", "pandas", "scikit-learn", "numpy", "matplotlib")
volume = modal.Volume.from_name("my-volume")
app = modal.App("stock-prediction")

#Our LSTM model module... A RNN 
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Use the last output of LSTM
        return output

# Data preparation and model training function
@app.function(gpu="A100:7", image=image, volumes={"/data": volume})
def train_lstm_on_modal():
    # Load data from the volume
    data = pd.read_csv("/data/AAPL_30_years.csv")

    # Convert Date column to datetime format (if not already)
    data["Date"] = pd.to_datetime(data["Date"], errors="coerce")

    # Convert numeric columns properly
    numeric_cols = ["Open", "High", "Low", "Close", "Volume", "News_Sentiment", "Economic Health"]
    for col in numeric_cols:
        data[col] = pd.to_numeric(data[col], errors="coerce")  # Force conversion, invalid values become NaN

    # Handle missing and infinite values
    if data.isna().any().any():
        print("Data contains NaN values. Filling missing values...")
        data.fillna(data.mean(), inplace=True)
    if np.isinf(data.select_dtypes(include=[np.number])).any().any():
        print("Data contains infinite values. Replacing with max value...")
        data.replace([np.inf, -np.inf], np.nan, inplace=True)
        data.fillna(data.mean(), inplace=True)

    # Normalize the numerical features
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['Open', 'High', 'Low', 'Close', 'Volume', 'Economic Health']] = scaler.fit_transform(
        data[['Open', 'High', 'Low', 'Close', 'Volume', 'Economic Health']])

    # Save the scaler
    joblib.dump(scaler, '/data/scaler.pkl')  # Save scaler to the volume

    # Prepare data for LSTM
    def create_sequences(data, seq_length):
        sequences, labels = [], []
        for i in range(len(data) - seq_length):
            seq_features = data.iloc[i:i+seq_length][['Open', 'High', 'Low', 'Close', 'Volume', 'News_Sentiment', 'Economic Health']].values
            label = data.iloc[i+seq_length]['Close']  # Predict the 'Close' price
            sequences.append(seq_features)
            labels.append(label)
        return np.array(sequences), np.array(labels)

    seq_length = 60  # Use the last 60 days as input
    X, y = create_sequences(data, seq_length)

    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Convert data to PyTorch tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

    # Create the LSTM model
    input_size = X_train.shape[2]  # Number of features
    hidden_size = 64
    output_size = 1
    model = LSTMModel(input_size, hidden_size, output_size).cuda()

    # Training setup
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    training_losses = []

    # Training loop
    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train.cuda())
        loss = criterion(outputs.squeeze(), y_train.cuda())
        loss.backward()
        optimizer.step()
        training_losses.append(loss.item())
        if epoch % 10 == 0:
            print(f'Epoch [{epoch+1}/50], Loss: {loss.item():.4f}')

    # After training and saving predictions, call analyze_predictions
    result = analyze_predictions()  # Call the analysis function
    print(result)  # Print the result for debugging
    
    # Save model
    torch.save(model.state_dict(), "/data/lstm_model.pt")

    # Save y_test for later comparison
    np.save('/data/y_test.npy', y_test.numpy())
    
    # Generate predictions for evaluation
    model.eval()
    with torch.no_grad():
        predictions = model(X_test.cuda()).cpu().numpy()
    np.save('/data/predictions.npy', predictions)  # Save predictions for analysis


def analyze_predictions():
    
    #Get scaler and test data
    scaler = joblib.load('/data/scaler.pkl')  # Load the saved scaler
    y_test = np.load('/data/y_test.npy')
    predictions = np.load('/data/predictions.npy')

    #Determine predictions
    y_test_inv = scaler.inverse_transform(np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 6 - 1)))))[:, 0]
    predictions_inv = scaler.inverse_transform(np.hstack((predictions, np.zeros((predictions.shape[0], 6 - 1)))))[:, 0]

    #Determine accuracy
    mae = np.mean(np.abs(predictions_inv - y_test_inv))
    rmse = np.sqrt(np.mean((predictions_inv - y_test_inv) ** 2))
    accuracy = 100 - (mae / np.mean(y_test_inv) * 100)  # Calculate accuracy percentage

    # Plot forecast vs actual
    plt.figure(figsize=(14, 7))
    plt.plot(y_test_inv, label='Actual Prices', color='blue')
    plt.plot(predictions_inv, label='Predicted Prices', color='red')
    plt.title("Stock Price Forecast vs Actual Prices")
    plt.legend()
    plt.show()

    #Analytics generated
    money_won_lost = np.sum(predictions_inv - y_test_inv)  # Total money gained or lost by the predictions

    return f"Analysis complete! MAE: {mae:.4f}, RMSE: {rmse:.4f}, Accuracy: {accuracy:.2f}%, Money Won/Lost: {money_won_lost:.2f}"

with modal.enable_output():
    with app.run():
        result = train_lstm_on_modal.remote()
       


Output()

✓ Initialized. View run at https://modal.com/apps/marlon-dominguez307/main/ap-vBBgbxukF5R1GKZbwZ8e93

Output()

✓ Created objects.
└── 🔨 Created function train_lstm_on_modal.

Output()

...

Epoch [41/50], Loss: 0.0005


Analysis complete! MAE: 2.4024, RMSE: 2.9645, Accuracy: 98.55%, Money Won/Lost: 194.33


Stopping app - local entrypoint completed.


Runner terminated.


✓ App completed. View run at https://modal.com/apps/marlon-dominguez307/main/ap-vBBgbxukF5R1GKZbwZ8e93